In [ ]:
import requests

def fetch_air_range(station_id, date_from, date_until):
    url = 'http://airviro.klab.ee/station/csv'
    data = {
        'filter[type]': 'INDICATOR',
        'filter[cancelSearch]': '',
        'filter[stationId]': station_id,
        'filter[dateFrom]': date_from,
        'filter[dateUntil]': date_until,
        'filter[submitHit]': '1',
        'filter[indicatorIds]': ''
    }
    response = requests.post(url, data)
    return response.text

import datetime

def get_first_and_last_day_of_month(year, month):
    # Get the first day of the month
    first_day = datetime.date(year, month, 1)

    # Get the number of days in the month
    if month == 12:
        num_days = 31
    else:
        num_days = (datetime.date(year, month+1, 1) - datetime.timedelta(days=1)).day

    # Get the last day of the month
    last_day = datetime.date(year, month, num_days)

    return first_day, last_day


Apply functions from the last cell to iterate over the moths of year 2022, fetch air quality infromation for each month and write it into a file.

In [ ]:
year = 2022
for month in range(1, 13):
    first_day, last_day = get_first_and_last_day_of_month(year, month)
    print(first_day, last_day)

    data = fetch_air_range(8, first_day, last_day)
    with open(f'air_data/air_{year}_{month}.csv', 'w') as f:
        f.write(data)

It would be cumbersome to check each individual month file. Stich them together into single year file.

In [ ]:
data = []
for month in range(1, 13):
    with open(f'air_data/air_{year}_{month}.csv', 'r') as f:
        if month == 1:
            header = f.readline()
            data.append(header)
        else:
            f.readline()
        data.extend(f.readlines())

with open(f'air_data/air_{year}.csv', 'w') as f:
    f.writelines(data)
